In [166]:
import os
import base64
import time
import dateutil.parser as parser
#from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import numpy as np

from email.header import decode_header
import re
import urllib.parse
import quopri
from io import BytesIO
import ast
import imaplib
import email
from dotenv import load_dotenv

load_dotenv('/Users/yushunli/env.')

#Load the user name and passwd from yaml file
user, password = os.getenv('GMAIL_USER'), os.getenv('GMAIL_PW')

import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
#URL for IMAP connection
imap_url = 'imap.gmail.com'

# Connection with GMAIL using SSL
my_mail = imaplib.IMAP4_SSL(imap_url)

# Log in using your credentials
my_mail.login(user, password)

# Select the Inbox to fetch messages
my_mail.select('Inbox')

('OK', [b'4199'])

In [4]:
#Define Key and Value for email search
#For other keys (criteria): https://gist.github.com/martinrusev/6121028#file-imap-search
key = 'FROM'
value = 'theffnewsletter@substack.com'
_, data = my_mail.search(None, key, value)  #Search for emails with specific key and value

mail_id_list = data[0].split()  #IDs of all emails that we want to fetch 

msgs = [] # empty list to capture all messages
#Iterate through messages and extract data into the msgs list
for num in mail_id_list:
    typ, data = my_mail.fetch(num, '(RFC822)') #RFC822 returns whole message (BODY fetches just body)
    msgs.append(data)

In [5]:
#convert msgs to a dataframe
df = pd.DataFrame(msgs, columns=['raw_message', 'random'])

In [31]:
subject = []
date = []
body = []
for msg in msgs[::-1]:
    for response_part in msg:
        if type(response_part) is tuple:
            my_msg=email.message_from_bytes((response_part[1]))
            #print("_________________________________________")
            #print ("subj:", my_msg['subject'])
            subject.append(my_msg['subject'])
            date.append(my_msg['date'])
            #print ("from:", my_msg['from'])
            for part in my_msg.walk():  
                #print(part.get_content_type())
                if part.get_content_type() == 'text/plain':
                    #print (part.get_payload())
                    body.append(part.get_payload())

#Remove the last element from the lists 'subject' and 'date'
subject.pop()
date.pop()

#Create an empty dataframe
df = pd.DataFrame()
#Fill the dataframe with the lists
df['subject'] = subject
df['date'] = date
df['body'] = body
#convert the 'date' column to datetime format in 'yyyy-mm-dd' format
df['date'] = pd.to_datetime(df['date']).dt.date

def decode_text(text):
    decoded_header = decode_header(text)
    decoded_parts = [part.decode(encoding or 'utf8') if not isinstance(part, str) else part for part, encoding in decoded_header]
    return ''.join(decoded_parts)

df['subject'] = df['subject'].apply(decode_text)
df['body'] = df['body'].apply(decode_text)
df.sort_values(by='date', ascending=True, inplace=True)
df.sample(10)

,subject,date,body
31,"""Were Brady and Brees even that good, or was i...",2021-09-30,View this post on the web at https://email.mg2...
16,57 Nuggets From Week 1,2022-09-15,View this post on the web at https://theffnews...
2,Anthony Richardson Going 1.01 in Fantasy Baske...,2023-07-13,View this post on the web at https://theffnews...
26,Michael Pittman Jr. Has Arrived,2021-11-04,View this post on the web at https://email.mg2...
22,'Welcome to the “fantasy football is dumb and ...,2021-12-09,View this post on the web at https://email.mg2...
9,"""Dynasty RB1 gets changed more frequently than...",2022-11-10,View this post on the web at https://theffnews...
18,"""Don't mind me, I'm just going to stare at the...",2022-09-01,View this post on the web at https://theffnews...
3,Greg Dulcich is Goated,2023-07-06,View this post on the web at https://theffnews...
0,Training Camp Spice,2023-07-27,View this post on the web at https://theffnews...
39,"Trey Lance, Tua Tagovailoa, and Donovan People...",2021-08-05,View this post on the web at https://email.mg2...


In [7]:
def clean_email_body(email_body):
    # Remove soft line breaks
    email_body = email_body.replace('=\r\n', '')

    # Replace unicode characters
    email_body = re.sub(r'=[0-9A-Fa-f]{2}', '', email_body)

    # Decode URLs
    url_pattern = r'(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)'
    urls = re.findall(url_pattern, email_body)
    for url in urls:
        decoded_url = urllib.parse.unquote(url)
        email_body = email_body.replace(url, decoded_url)

    # Replace =20 with a space
    email_body = email_body.replace('=20', ' ')

    return email_body


In [8]:
def decode_quoted_printable(encoded_str):
    decoded_bytes = quopri.decodestring(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    # Replace "\r\n" with "\n"
    decoded_str = decoded_str.replace('\r\n', '\n')
    return decoded_str

In [14]:
def remove_http_links_in_brackets(text):
    # The regular expression pattern below matches any text that:
    # - starts with a '[' (escaped as '\[')
    # - is followed by any number of any characters (matched by '.*?')
    # - contains 'http'
    # - ends with a ']' (escaped as '\]')
    return re.sub(r'\[.*?http.*?\]', '(Player Highlighted)', text)

In [34]:
def remove_intro_conclusion(text):
    try:
        # Remove introduction
        intro_keyword = 'Welcome back!'
        text = text.split(intro_keyword, 1)[1]  # Keep only the text after the intro_keyword

        # Remove conclusion
        conclusion_keyword = 'Unsubscribe'
        text = text.rsplit(conclusion_keyword, 1)[0]  # Keep only the text before the conclusion_keyword

        return text.strip()  # Remove leading and trailing whitespace

    except IndexError:  # Handle case where keywords are not found
        return text
    
    import re

def remove_intro_conclusion2(text):
    # Remove intro
    intro_pattern = r'^(.*?As always, you can join the discussion on /r/fantasyfootball \(Player Highlighted\) and /r/DynastyFF \(Player Highlighted\))'
    text = re.sub(intro_pattern, '', text, flags=re.DOTALL)
    
    # Remove conclusion
    conclusion_pattern = r'(That’s it for now. Join our community on Patreon \(Player Highlighted\) and get all our rankings and access to our Discord chat! You can also follow me on Twitter \(Player Highlighted\) and Instagram \(Player Highlighted\) where I browse over 1,000 analysts and beat writers and retweet/post all the good stuff..*)$'
    text = re.sub(conclusion_pattern, '', text, flags=re.DOTALL)
    
    return text


In [35]:
cleaned = []

for body in df.body:
    text = decode_quoted_printable(body)
    text = remove_http_links_in_brackets(text)
    text = remove_intro_conclusion(text)
    text = remove_intro_conclusion2(text)
    cleaned.append(text)

df['cleaned_body'] = cleaned
df.sample(10)

,subject,date,body,cleaned_body
4,All your 1st round rookies are about to suck,2023-06-29,View this post on the web at https://theffnews...,This is one of the slower periods of the offse...
1,"My boss gave me the franchise tag, so I held o...",2023-07-20,View this post on the web at https://theffnews...,Training camps are so close we can taste it. ...
37,3 Weeks Until Kickoff!,2021-08-19,View this post on the web at https://email.mg2...,\nNews and Notes:\nKerryon Johnson (Player Hig...
11,"🏈 ""Through seven weeks, Daniel Jones has more ...",2022-10-27,View this post on the web at https://theffnews...,I propose we invent a new version of fantasy f...
12,"""Russell Wilson has seven more bathrooms in hi...",2022-10-20,View this post on the web at https://theffnews...,"We’re 1/3 through the season, so some of us ar..."
29,"Hope you're ready for Kadarius, Hollywood, Sch...",2021-10-14,View this post on the web at https://email.mg2...,We’re getting excited by about half the NFL th...
21,"""Next year when the fantasy football playoffs ...",2021-12-23,View this post on the web at https://email.mg2...,View this post on the web at https://email.mg2...
10,"""Zach Wilson is like if Patrick Mahomes played...",2022-11-03,View this post on the web at https://theffnews...,Whew. It was a week of trades for the record b...
18,"""Don't mind me, I'm just going to stare at the...",2022-09-01,View this post on the web at https://theffnews...,"In just one short week, we’ll have real footba..."
17,I compile a quick/easy summary of the best FF ...,2022-09-08,View this post on the web at https://theffnews...,We made it. Real football begins today. Let’s ...


In [36]:
for i in df.cleaned_body.sample(3):
    print(i)
    print('_______________________________________________________________')

Training camps are so close we can taste it.  Here’s your last week of headlines and analysis that are unadulterated by the sensationalist feast we’re all about to partake in.
Now let’s get to it.
Analysis & Hype:
Since 2019 when Deebo entered the league:
Deebo Samuel:
• 340 targets
• 123 carries
Austin Ekeler (Player Highlighted)
• 394 targets
• 658 carries
Who’s the real “wide back”?
Nico Collins (Player Highlighted) in 2021: 
• 13% Target Share 
• 16% Target per Route Run
Nico Collins in 2022: 
• 20% Target Share 
• 25% Target Per Route Run
RB leaders by yards after contact per attempt (2022 - min. 100 touches)
1 Tony Pollard 
2 Nick Chubb 
3 Tyler Allgeier 
4 D’Andre Swift (Player Highlighted)
RB leaders by missed tackles forced per touch (2022 - min. 100 touches) 
1 David Montgomery 
2 Nick Chubb 
3 Rhamondre Stevenson 
4 D'Andre Swift
Rhamondre Stevenson (Player Highlighted) is one of only five running backs over the last five years to have 69+ receptions and average 4.9 yards pe

In [37]:
#Check the value counts of the 'year' and 'month' from the 'date' column
df['season'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
#Create a week of the month column by ranking all of the rows for that given month and year combination, and giving the earliest date a rank of 1, the second earliest a rank of 2, etc.
df['week'] = df.groupby(['season', 'month'])['date'].rank(method='first')
#Create another column called 'Type' where the value is 'Regular' if the month is between 9 and 12
df['type'] = df['date'].apply(lambda x: 'in_season' if x.month >= 9 else 'preseason')
#If the row is the first row or week of the month 'September', then the value is 'preseason'
df.loc[(df['month'] == 9) & (df['week'] == 1), 'type'] = 'preseason'
#Drop the 'month' column
df.drop(['month', 'week'], axis=1, inplace=True)
df = df[['season', 'date', 'type', 'subject', 'cleaned_body']]

In [38]:
sample = (df.loc[(df['type'] == 'preseason') & (df['season'] != 2023)])
sample.reset_index(inplace=True, drop=True)

Newsletters sections:

Good:
- Analysis and Hype
- Camp Buzz/Hype/Fluff

Bad:
- Cold Water

Neutral:
- Articles and Analysis
- News & Notes
- Coachspeak and Beat Reports:

Ignore:
- The Fun Stuff

In [39]:
print(sample.cleaned_body[2])


News & Notes:
Carson Wentz (Player Highlighted) trending to open the season as the starter with All-Pro G Quenton Nelson on same path after foot surgeries.
Bills and Josh Allen (Player Highlighted) have reached agreement on a 6-year, $258M extension, locking him in through 2028. $150M guaranteed.
Ravens rookie WR Rashod Bateman (Player Highlighted) suffered a “somewhat significant” groin injury. Has a chance to be ready for Week 1.
D'Andre Swift (Player Highlighted) has missed "significant practice time while nursing a sore groin" during training camp.
DJ Chark Jr. (Player Highlighted) had a minor break on his hand, and underwent surgery. Urban Meyer says he will be back by Week 1.
Coachspeak and Beat Reports:
The Jerry Jeudy hype has been gaining a lot of steam in recent weeks. I’m seeing these kinds of quotes from almost all the Denver beat writers consistently:
“Theme of 2021: Jerry Jeudy (Player Highlighted) torching DBs.”
“It almost seems unfair at this point to ask anyone to gua

## Sentiment Analysis via GPT4

In [40]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [127]:

prompt = "You are a Fantasy Football AI analyst tasked with analyzing the content of a newsletters for sentiment analysis. The input text will contains fantasy related news and analysis on NFL players. I've already highlighted the players of focus in the text with the characters \"(Player Highlighted)\" following their names. There will often be other players that this highlighted player is being compared to, but I'm not interested in them, just the \"Player Highlighted\". \n\nYou will go through the text and analyze each player highlighted and output the analysis in a list of python dicts with the following format: [{'Player': Player_Name, 'Type': Type, 'Sentiment': score}, etc.]\n\nType will be a short descriptor of the analysis. Only select one of the following categoryies: Performance, Injury, Contract, Roster, Other\n\nSentiment will measure the sentiment of the analysis based on a scale of -5 to 5 where 5 is the best positive praise, -5 is the worst negative criticism, and 0 is a very neutral statement"


In [128]:
def sentiment_analyzer(df, prompt = prompt):
    """ 
    This function will use GPT-4 to analyze the sentiment of the newsletter.
    """
    
    #Create a empty list to store the results
    results = []

    #For loop to iterate through the dataframe
    for i in range(len(df)):
        season = df['season'][i]
        text = df.cleaned_body[i]
        messages =   [
                        {
                        "role": "system",
                        "content": prompt
                        },
                        {
                        "role": "user",
                        "content": text
                        }
                    ]
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0) 
        #Convert the results
        response = response.choices[0].message["content"]
        #convert the response to a list
        #results = results.split("\n")
        # Convert each string element to a dictionary using ast.literal_eval()
        #list_of_dicts = [ast.literal_eval(item.rstrip(',')) for item in results]

        # Convert the list of dictionaries into a DataFrame
        #result_df = pd.DataFrame(results)
        #result_df['season'] = season
        results.append(response)
        #Print the progress
        print(f"{i+1} articles completed")
    
    return results
        

In [97]:
results = sentiment_analyzer(sample)

1 articles completed
2 articles completed
3 articles completed
4 articles completed
5 articles completed
6 articles completed
7 articles completed
8 articles completed
9 articles completed


In [124]:
dfs = []
for i in range(len(results)):
    test = results[i]
    season = seasons[i]
    test = re.sub(r"Ty'Son Williams", "Tyson Williams", test)
    #Remove the 1st and last character from test
    test = test[1:-1]
    data = test.strip()
    data = '[' + data + ']'
    # Use ast.literal_eval to convert the string to a list of dictionaries
    list_of_dicts = ast.literal_eval(data)
    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(list_of_dicts)
    df['season'] = season
    dfs.append(df)

# Concatenate all the DataFrames in the list
sentiment_df = pd.concat(dfs)
sentiment_df.head()

,Player,Type,Sentiment,season
0,Aaron Rodgers,Training Camp Attendance,3,2021
1,Michael Thomas,Injury,-4,2021
2,Randall Cobb,Trade,2,2021
3,Dak Prescott,Injury,-3,2021
4,Kelvin Benjamin,Release,-3,2021


In [138]:
#Remove rows if the type contains any of the following strings: 'injury', 'contract', 'trade', 'suspen' or 'humor'
sentiment_df2 = sentiment_df[~sentiment_df['Type'].str.contains('injury|contract|trade|suspen|humor|release', case=False)]
sentiment_df2.head()

,Player,Type,Sentiment,season
0,Aaron Rodgers,Training Camp Attendance,3,2021
5,Odell Beckham Jr.,Physical Fitness,3,2021
8,Jerry Jeudy,Performance,4,2021
9,Brandon Aiyuk,Performance,4,2021
10,Joe Burrow,Performance,4,2021


### Analyze Performance

In [156]:
#Group by Player and season, and create 2 agg columns: sum and mean
df_newsletter = sentiment_df2.groupby(['Player', 'season'], as_index=False).agg({'Sentiment': ['sum', 'mean']})
# Flattening the MultiIndex columns
df_newsletter.columns = ['_'.join(col).strip() for col in df_newsletter.columns.values]

# Renaming the columns to remove the trailing underscore for Player and season
df_newsletter.rename(columns={"Player_": "Player", "season_": "season"}, inplace=True)
df_newsletter.sort_values(by=[('Sentiment_sum')], ascending=False).head(15)

,Player,season,Sentiment_sum,Sentiment_mean
123,Kyle Pitts,2022,15,5.000000
45,Damien Harris,2021,15,3.750000
23,Bryan Edwards,2021,15,3.750000
44,Dameon Pierce,2022,13,4.333333
94,Jerry Jeudy,2021,13,4.333333
18,Brandon Aiyuk,2021,13,4.333333
100,Jonathan Taylor,2021,13,4.333333
68,George Pickens,2022,13,4.333333
108,Justin Fields,2021,11,3.666667
2,Aaron Rodgers,2021,11,2.750000


In [150]:
performance = pd.read_csv('/Users/yushunli/Documents/Data Science/jupyter_notebook/Fantasy Football/player_data.csv')
#Rename column 'player_name' to 'Player'
performance.rename(columns={'player_name':'Player'}, inplace=True)

In [158]:
#Flatten the df_newsletter dataframe 'Sentiment' section
output = df_newsletter.merge(performance, on = ['Player', 'season'], how = 'left') 
#Show output rows where 'PPG' is not null
output[output['PPG'].notnull()]

,Player,season,Sentiment_sum,Sentiment_mean,overallPickNumber,playerId,position,keeper,teamId,owner,draft_position_rank,budget_before,budget_after,bidAmount,Total_Points,Game_Played,PPG,actual_position_rank,bidAmount_bucket
0,A.J. Brown,2022,5,5.00,22.0,4047646.0,WR,False,13.0,Kai Shen,7.0,192.0,157.0,35.0,175.50,12.0,14.6,5.0,35-40
2,Aaron Rodgers,2021,11,2.75,39.0,8439.0,QB,False,13.0,Kai Shen,2.0,64.0,54.0,10.0,224.82,11.0,20.4,9.0,10-15
6,Alec Pierce,2022,8,4.00,207.0,4360078.0,WR,False,2.0,Francis Jin,27.0,3.0,2.0,1.0,79.00,12.0,6.6,44.0,0-5
7,Allen Lazard,2022,3,3.00,128.0,3128390.0,WR,False,15.0,Kyle O'Meara,22.0,11.0,5.0,6.0,114.50,11.0,10.4,25.0,5-10
9,Alvin Kamara,2021,10,5.00,20.0,3054850.0,RB,False,15.0,Kyle O'Meara,4.0,128.0,49.0,79.0,142.00,8.0,17.8,14.0,75-80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Tua Tagovailoa,2021,8,4.00,133.0,4241479.0,QB,False,15.0,Kyle O'Meara,7.0,9.0,8.0,1.0,140.70,8.0,17.6,15.0,0-5
187,Tyler Allgeier,2022,4,4.00,175.0,4373626.0,RB,False,14.0,Andrew Kim,33.0,2.0,1.0,1.0,86.10,12.0,7.2,35.0,0-5
188,Tyler Lockett,2021,3,3.00,59.0,2577327.0,WR,False,7.0,Scott Lin,10.0,99.0,75.0,24.0,140.60,12.0,11.7,21.0,20-25
189,Tyreek Hill,2021,3,3.00,16.0,3116406.0,WR,False,5.0,John Dong,2.0,175.0,119.0,56.0,193.80,12.0,16.2,4.0,55-60


In [161]:
df_newsletter.to_csv('ff_newsletter.csv', index=False)

In [172]:

sample_df.head()

,Player,Position,Sentiment_sum,Difference,actual_position_rank
0,A.J. Brown,WR,5,2,5.0
1,Aaron Rodgers,QB,11,-7,9.0
2,Alec Pierce,WR,8,-17,44.0
3,Allen Lazard,WR,3,-3,25.0
4,Alvin Kamara,RB,10,-10,14.0


In [177]:
sample_df = pd.read_csv('/Users/yushunli/Documents/Data Science/jupyter_notebook/Fantasy Football/ffnewsletter.csv')
#Filter for rows where 'actual_position_rank' is not null
sample_df = sample_df[sample_df['actual_position_rank'].notnull()]
sample_df['actual_position_rank2'] = 1 / sample_df['actual_position_rank']
for i in sample_df['Position'].unique():
    #Create a new column for inverse of 'actual_position_rank'
    #Plot the scatter plot using plotly and set the 'Player' as the hover text, use the inverse of 'actual_position_rank' as size
    fig = px.scatter(sample_df[sample_df['Position'] == i], x='Sentiment_sum', y='Difference', hover_name='Player', size='actual_position_rank2', title='Sentiment_sum vs Difference for ' + i)
    #Plot a black line at y = 0
    fig.add_shape(type='line', x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=2))
    #Plot a black line at x = 0
    fig.add_shape(type='line', x0=0, y0=-1, x1=0, y1=1, line=dict(color='black', width=2))
    #Plot a regression line
    #fig.add_trace(go.Scatter(x=sample_df[sample_df['Position'] == i]['Sentiment_sum'], y=sample_df[sample_df['Position'] == i]['Difference'], mode='lines', name='Regression Line'))
    #Show the plot
    fig.show()


In [179]:
#Calculate the correlation between 'Sentiment_sum' and 'Difference' from the sample_df
for i in sample_df.Position.unique():
    subset = sample_df[sample_df['Position'] == i]
    #Calculate the correlation between 'Sentiment_sum' and 'Difference' from the subset and print the results
    print(i, subset['Sentiment_sum'].corr(subset['Difference']))

WR 0.047880092719984425
QB -0.18859727265383266
RB 0.06514825247809901
TE 0.20470395978399825
